In [1]:
import gensim
import os
import codecs
import pandas as pd 
import numpy as np

C:\Users\GoldFire\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
print('Loading models...')
model = gensim.models.Word2Vec.load('./models/idwiki_word2vec.model')
modelOOV = gensim.models.Word2Vec.load('./models/modelWord2Vec_OOV.model')

Loading models...


In [6]:
print('Reading training pairs...')
word_pairs = codecs.open('corpus_TBK-BK.csv', 'r', 'utf-8')

pairs = pd.read_csv(word_pairs)

Reading training pairs...


In [8]:
missing = 0
for n in range (len(pairs)):
	if pairs['source'][n] not in modelOOV.wv.vocab or pairs['target'][n] not in model.wv.vocab:
		missing = missing + 1
		pairs = pairs.drop(n)

pairs = pairs.reset_index(drop = True)
print(pairs)

        source      target
0        abjat       abjad
1           da         ada
2           ad         ada
3         adek        adik
4      adpokat     advokat
5        afdol       afdal
6            z         aja
7          ajk        ajak
8        ahlak      akhlak
9        aktip       aktif
10   aktifitas   aktivitas
11         ane         aku
12          aq         aku
13         aqu         aku
14         eke         aku
15        eyke         aku
16         gua         aku
17          ak         aku
18           q         aku
19        almt      alamat
20         alm    almarhum
21     ambeyen     ambeien
22     ambulan    ambulans
23   amandemen   amendemen
24     amphibi      amfibi
25       amvun       ampun
26         ank        anak
27     analisa    analisis
28      handal       andal
29      anggep      anggap
..         ...         ...
856       univ  universita
857        tuk       untuk
858        utk       untuk
859     urgent       urgen
860     ngurus        urus
8

In [9]:
pairs['vector_source'] = [modelOOV[pairs['source'][n]] for n in range (len(pairs))]
pairs['vector_target'] = [model[pairs['target'][n]] for n in range (len(pairs))]

# # first 5000 from both languages, to train translation matrix
source_training_set = pairs['vector_source'][:5000]
target_training_set = pairs['vector_target'][:5000]

matrix_train_source = pd.DataFrame(source_training_set.tolist()).values
matrix_train_target = pd.DataFrame(target_training_set.tolist()).values

print('Generating translation matrix')

# # Matrix W is given in  http://stackoverflow.com/questions/27980159/fit-a-linear-transformation-in-python
translation_matrix = np.linalg.pinv(matrix_train_source).dot(matrix_train_target).T
print ('display translation')

C:\Users\GoldFire\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
C:\Users\GoldFire\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Generating translation matrix
display translation


In [10]:
def most_similar_vector(self, vectenter, topn=5):
    self.init_sims()
    dists = np.dot(self.syn0norm, vectenter)
    if not topn:
        return dists
    best = np.argsort(dists)[::-1][:topn ]
        # ignore (don't return) words from the input
    result = [(self.index2word[sim], float(dists[sim])) for sim in best]
    return result[:topn]

def top_translations(w,numb=5):
    val = most_similar_vector(model.wv,translation_matrix.dot(model[w]),numb)
    #print 'traducwithscofres ', val
    return val


def top_translations_list(w, numb=5):
    val = [top_translations(w,numb)[k][0] for k in range(numb)]
    return val

def saveModelTranslationMatrix():
    pass

temp = 1
#top_matches = [ pairs['target'][n] in top_translations_list(pairs['source'][n]) for n in range(5000,5003)] 

# print out source word and translation
def display_translations():
    for word_num in range(range_start, range_end):
        source_word = pairs['source'][word_num]
        translations = top_translations_list(pairs['target'][word_num]) 
        print(source_word, translations)

In [29]:
def testNormalization(word):
    for i, elem in enumerate(pairs['source'][:]):
        if word in elem:
            indices = i
            return top_translations_list(pairs['target'][indices])
        if i == len(pairs['source'][:])-1:
            return "tidak ditemukan"

In [31]:
testNormalization("cewe")

C:\Users\GoldFire\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
C:\Users\GoldFire\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  This is separate from the ipykernel package so we can avoid doing imports until


['sangat', 'laun', 'amat', 'paling', 'amatlah']